In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pyodbc
import cx_Oracle

from lifelines.plotting import plot_lifetimes      # Lifeline package for the Survival Analysis
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
figsize(12,8)

In [3]:
ip = '172.16.80.74'
port = '1521'
SID = 'RPTDB'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
connection = cx_Oracle.connect('ANLYUSR', 'ANLYUSR$', dsn_tns)

query = "SELECT /*+ PARALLEL(10) */ \
TB1.AGMTNO,TB1.AGMT_DATE,TB1.PORTFOLIO,TB1.CHANNEL, \
TB1.PAYMENT_TYPE,TB1.AREA_CODE,TB1.DEALER_TYPE,TB1.DEALER_CODE,TB1.MAINDEALER_CODE, \
TB1.EXECUTIVE_CODE,TB1.PRODUCT_CODE,TB1.MODEL_CODE,TB1.SCHEME_TYPE, \
TB1.SCHEME_CODE,TB1.ADVANCE_EMI_COUNT,TB1.TENUR,TB1.ROI, \
TB1.EMI,TB1.PORCESSING_FEE,TB1.STAMPING_CHARGES,TB1.ASSET_COST, \
TB1.LOAN_AMOUNT,TB1.DOWN_PAYMENT,TB1.INITITAL_HIRE,TB1.LTV,TB1.AMORT_IRR,TB1.CUSTOMER_IRR, \
TB1.NET_IRR,TB1.VECHICAL_INSURANCE_AMT,TB1.SPECIAL_INSURANCE,TB1.SPECIAL_INSURANCE_FOR, \
TB1.SPECIAL_INSURANCE_PRODUCT,TB1.SPECIAL_INSURANCE_AMOUNT, \
TB1.GENDER,TB1.PANNO,TB1.DRIVING_LICENSE,TB1.PASSPORT,TB1.AADHAR,TB1.RATION, \
TB1.EXISTING_CUSTOMER,TB1.EMPLOYMENT_TYPE,TB1.CUSTOMER_PROFILE_CODE,TB1.RESIDENT_TYPE, \
TB1.BANK_ACCOUNT,TB1.PINCODE,TB1.CITY,TB1.TALUK,TB1.TOWN,TB1.STATE, \
TB1.TRN_MONTH,TB1.TRN_YEAR,TB1.FIN_YEAR,TB1.PRODUCT_TYPE,TB1.ACTUAL_LOAN_AMOUNT, \
TB1.PRODUCT_CUST_IRR,TB1.PRODUCT_AMORT_IRR,TB1.PRODUCT_TENUR,TB1.PF_SERVICE_TAX, \
TB1.SUB_LOCATION,TB1.MICRO_MARKET,TB1.CIBIL_SCORE,TB1.COBORROWER, \
TB1.GUARANTOR,TB1.RELIGION,TB1.CIBIL_HIT,TB1.CIBIL_OVERDUE, \
TB1.CIBIL_SCORE_COBORROWER,TB1.CIBIL_SCORE_GURANTOR, \
TB1.LAST_EMI_DATE - TB1.FIRST_EMI_DATE as duration_days, \
TB1.STATUS,TB1.ADDRESS_TYPE, \
TB1.ADDRESS_SLNO,TB1.SUB_STATUS,TB2.MOB, \
round((TB1.AGMT_DATE - TB1.DOB)/365) as AGE FROM TBL_TRN_CUSTOMER_DETAILS TB1 LEFT OUTER JOIN \
(SELECT TBL1.AGMTNO,TBL1.MOB,TBL1.CLOSING_BUCKET FROM \
(SELECT AGMTNO,MOB,CLOSING_BUCKET, \
DENSE_RANK() OVER (PARTITION BY AGMTNO ORDER BY MOB) AS MOBRANK \
FROM TBL_TRN_COLLECTION_DETAILS \
WHERE CLOSING_BUCKET = 1 \
AND MOB <> 0) TBL1 \
where TBL1.mobrank = 1) TB2 \
ON TB1.AGMTNO = TB2.AGMTNO \
AND TB1.AGMTNO is not null \
AND TB1.PORTFOLIO = 'TW' \
AND TB1.CHASSIS_NUMBER is not null"

In [4]:
try:
    del Data
    del data
except:
    print("No data to delete")
import gc
gc.collect()

No data to delete


11

In [5]:
Data = pd.read_sql(query, con=connection, chunksize=50000)

In [6]:
data = pd.concat([chunk for chunk in Data])
#data = pd.read_csv("D:\cust_bounce_data_v1.csv")

KeyboardInterrupt: decoding with 'WINDOWS-1252' codec failed (KeyboardInterrupt: )

In [ ]:
data.head()

In [ ]:
data.info(max_cols=300)

In [ ]:
data.to_csv("D:\cust_bounce_data_v2.csv")

In [ ]:
data.isnull().sum()

In [ ]:
for i in data.columns:
    #if data[i].isnull().sum() != 0:
    print("column {} has ---unique----> {} ---null----> {} ".format(i,data[i].unique().size, data[i].isnull().sum()))

In [99]:
data = data[['AGMTNO', 'AGMT_DATE', 'PORTFOLIO', 'CHANNEL', 'PAYMENT_TYPE',
       'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE', 'MAINDEALER_CODE',
       'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE', 'SCHEME_TYPE',
       'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI', 'EMI',
       'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST', 'LOAN_AMOUNT',
       'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR', 'CUSTOMER_IRR',
       'NET_IRR', 'VECHICAL_INSURANCE_AMT', 'SPECIAL_INSURANCE',
       'SPECIAL_INSURANCE_FOR', 'SPECIAL_INSURANCE_PRODUCT',
       'SPECIAL_INSURANCE_AMOUNT', 'GENDER', 'PANNO', 'DRIVING_LICENSE',
       'PASSPORT', 'AADHAR', 'RATION', 'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE',
       'CUSTOMER_PROFILE_CODE', 'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE',
       'CITY', 'STATE', 'TRN_MONTH', 'TRN_YEAR', 'FIN_YEAR',
       'ACTUAL_LOAN_AMOUNT', 'PRODUCT_CUST_IRR',
       'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR', 'PF_SERVICE_TAX',
       'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'LAST_EMI_DATE','FIRST_EMI_DATE', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'DOB']]

In [104]:
data = data[data.CHANNEL.notnull()]

In [107]:
data = data[data.MODEL_CODE.notnull()]
data = data[data.SCHEME_TYPE.notnull()]
data = data[data.DOWN_PAYMENT.notnull()]
data = data[data.ROI.notnull()]
data = data[data.GENDER.notnull()]
data = data[data.EMPLOYMENT_TYPE.notnull()]
data = data[data.CUSTOMER_PROFILE_CODE.notnull()]
data = data[data.CITY.notnull()]
data = data[data.RELIGION.notnull()]

In [111]:
data['VECHICAL_INSURANCE_AMT'].fillna(0, inplace=True)
data['SPECIAL_INSURANCE_FOR'].fillna('N/A', inplace=True)
data['SPECIAL_INSURANCE_PRODUCT'].fillna('N/A', inplace=True)
data['SPECIAL_INSURANCE_AMOUNT'].fillna(0, inplace=True)
data['EXISTING_CUSTOMER'].fillna('N/A', inplace=True)
data['RESIDENT_TYPE'].fillna('N/A', inplace=True)
data['CIBIL_SCORE'].fillna(0, inplace=True)
data['CIBIL_HIT'].fillna('N/A', inplace=True)
data['CIBIL_SCORE_COBORROWER'].fillna(0, inplace=True)
data['CIBIL_SCORE_GURANTOR'].fillna(0, inplace=True)
data['SUB_STATUS'].fillna('L', inplace=True)

In [113]:
data = data[data.AGE.notnull()]

In [130]:
#df['ID'] = pd.factorize(df.SrcIP)[0]

In [ ]:
data['trgt'] = data.

In [121]:
from pandas.api.types import is_string_dtype

In [124]:
for i in data.columns:
    if is_string_dtype(data[i]):
        data[i] = pd.factorize(data[i])[0]

In [129]:
data_v1 = data[['PORTFOLIO', 'CHANNEL', 'PAYMENT_TYPE',
       'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE', 'MAINDEALER_CODE',
       'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE', 'SCHEME_TYPE',
       'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI', 'EMI',
       'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST', 'LOAN_AMOUNT',
       'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR', 'CUSTOMER_IRR',
       'NET_IRR', 'VECHICAL_INSURANCE_AMT', 'SPECIAL_INSURANCE',
       'SPECIAL_INSURANCE_FOR', 'SPECIAL_INSURANCE_PRODUCT',
       'SPECIAL_INSURANCE_AMOUNT', 'GENDER', 'PANNO', 'DRIVING_LICENSE',
       'PASSPORT', 'AADHAR', 'RATION', 'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE',
       'CUSTOMER_PROFILE_CODE', 'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE',
       'CITY', 'STATE', 'TRN_MONTH', 'TRN_YEAR', 'FIN_YEAR',
       'ACTUAL_LOAN_AMOUNT', 'PRODUCT_CUST_IRR',
       'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR', 'PF_SERVICE_TAX',
       'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'DURATION_DAYS', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'FIRST_BOUNCE_MOB', 'AGE']]

In [127]:
from lifelines import CoxPHFitter

In [ ]:
# Using Cox Proportional Hazards model
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
cph.fit(data_v1, 'tenure', event_col='Churn')   ## Fit the data to train the model
cph.print_summary()    ## HAve a look at the significance of the features